In [8]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import _LRScheduler
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [9]:
smiles = 'Cc1ccccc1'
mol = Chem.MolFromSmiles(smiles)


In [5]:
fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)

In [7]:
fp.ToList()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [11]:
MORGAN_RADIUS = 2
MORGAN_NUM_BITS = 1024

def morgan_binary_features_generator(mol,
                                     radius: int = 2,
                                     num_bits: int = 1024) -> np.ndarray:
    """
    Generates a binary Morgan fingerprint for a molecule.

    Parameters
    ----------
    mol: A molecule (i.e. either a SMILES string or an RDKit molecule).
    radius: Morgan fingerprint radius.
    num_bits: Number of bits in Morgan fingerprint.

    Returns
    -------
    An 1-D numpy array containing the binary Morgan fingerprint.
    """
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    features_vec = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=num_bits)
    features = np.zeros((1,))
    DataStructs.ConvertToNumpyArray(features_vec, features)

    return features


In [12]:
f = morgan_binary_features_generator(smiles)

In [14]:
(f==0).all()

False

In [4]:
import inspect
from dataclasses import dataclass
@dataclass
class MolGraphAttrs:
    n_atoms = None
    n_bonds = None
    f_atoms = None
    f_bonds = None
    a2b = None
    b2a = None
    b2revb = None

In [5]:
inst = MolGraphAttrs()

In [6]:
inspect.getmembers(inst)

[('__class__', __main__.MolGraphAttrs),
 ('__dataclass_fields__', {}),
 ('__dataclass_params__',
  _DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False)),
 ('__delattr__',
  <method-wrapper '__delattr__' of MolGraphAttrs object at 0x12edf4fd0>),
 ('__dict__', {}),
 ('__dir__', <function MolGraphAttrs.__dir__()>),
 ('__doc__', 'MolGraphAttrs()'),
 ('__eq__', <bound method __create_fn__.<locals>.__eq__ of MolGraphAttrs()>),
 ('__format__', <function MolGraphAttrs.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of MolGraphAttrs object at 0x12edf4fd0>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of MolGraphAttrs object at 0x12edf4fd0>),
 ('__gt__', <method-wrapper '__gt__' of MolGraphAttrs object at 0x12edf4fd0>),
 ('__hash__', None),
 ('__init__',
  <bound method __create_fn__.<locals>.__init__ of MolGraphAttrs()>),
 ('__init_subclass__', <function MolGraphAttrs.__init_subclass__>),
 ('__le__', <method-wrapper '__le__' 

In [10]:
dir(inst)

['__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'a2b',
 'b2a',
 'b2revb',
 'f_atoms',
 'f_bonds',
 'n_atoms',
 'n_bonds']